In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import *
os.chdir(proj_root())


In [15]:
from spot.experiments.utils import remove_newer_syntax_for_repo, Path
from spot.experiments.type4py import Type4PySupportedSyntax
from spot.utils import get_dataset_dir
import shutil

# dataset_name = "InferTypes4Py"
dataset_name = "ManyTypes4Py"
repos_dir = get_dataset_dir(dataset_name) / "repos"
shutil.rmtree(repos_dir / "test-hityper", ignore_errors=True)
shutil.copytree(repos_dir / "test", repos_dir / "test-hityper")
remove_newer_syntax_for_repo(repos_dir / "test-hityper", Type4PySupportedSyntax)

Removing newer syntax: 100%|██████████| 1594/1594 [00:01<00:00, 873.90it/s] 


1594 / 1594 files have been rewritten.


In [16]:
from spot.experiments.hityper import run_hityper, eval_hityper_on_repos

test_repos = [
    p
    for p in (repos_dir / "test-hityper").iterdir()
    if p.is_dir()
]

hityper_path = Path("/home/jiayi/anaconda3/envs/hityper/bin/python")
out_dir = Path("output/hityper")
cache = PickleCache(Path(f"caches/run_hityper"))

eval_r = cache.cached(f"{dataset_name}.pkl", lambda: eval_hityper_on_repos(test_repos, hityper_path, out_dir, max_workers=4))

In [17]:
from spot.static_analysis import SignatureErrorAnalysis, AccuracyMetric
from spot.experiments.typet5 import accs_as_table_row, ModelWrapper


common_names = ModelWrapper.load_common_type_names(
    get_model_dir() / "model-v7--TrainingConfig(drop_env_types=False)"
)
metrics = AccuracyMetric.default_metrics(common_type_names=common_names)
# acc_metric = AccuracyMetric(common_type_names=ubiq_names)

accs = {
    m.name: SignatureErrorAnalysis(
        eval_r.pred_maps,
        eval_r.label_maps,
        m,
        error_on_mismatched_signature=False,
    ).accuracies
    for m in metrics
}

accs_as_table_row(accs)
print("-" * 40)
pretty_print_dict(accs)

Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
42.53 & 41.95 & 44.86 & 19.02 & 47.88
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
59.20 & 54.28 & 57.70 & 26.44 & 59.01
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
10.30 & 25.51 & 27.59 & 9.79 & 29.33
----------------------------------------
full_acc:
   full_acc: 42.53% (count=10.5k)
   full_acc_by_cat:
      FuncArg: 26.80% (count=5.9k)
      FuncReturn: 63.41% (count=4.5k)
      GlobalVar: 18.84% (count=69)
   full_acc_by_simple:
      complex: 14.59% (count=1.7k)
      simple: 47.85% (count=8.8k)
   full_acc_label_size: 1.3993
   full_acc_pred_size: 1.3325
   full_acc_ignored_labels: 0
   n_skipped_types: 3293
   n_missing_types: 1922
full_acc_common:
   full_acc_common: 59.20% (count=6.9k)
   full_acc_common_b